## 5 Analysis Step #3; PileUp

### 5.1 Pileup Data Format
The output of the mapping and marking of the duplicate reads step is in a non-human readable form (binary data is only readable by computers). Here, the output is transformed to a more human readable format. We will run a program that gets the mapped and duplicate removed data and describe for each mapped position what the base-pair information is. This program will go over the mapping and report the following details for each position that was mapped: 
* the chromosome name [1], 
* reference position [2], 
* reference base [3], 
* number of supporting reads [4],
* the match [5], either: 
    * a dot for a match in the forward strand or 
    * a comma for a match in the reverse strand
* the quality [6] (taken from the FASTQ file) can be seen in the last column.
```
1		2	   3	4	5	 6  
chr1	 10004   c	1	,	 [  
chr1     10005   g    3    ,,.   ?1>
```

The reason this format is called *pileup* is because for each *position* it will pile (or stack) the data from all mapped reads for that position. In the first example line above there was only one read with a match (the reference is a C as shown in column 3) in the reverse strand and the second line has three supporting reads with a match. See below for some examples where multiple reads are mapped on the same position.

<!--
pileup format: http://samtools.sourceforge.net/pileup.shtml   
-->

Choose the <strong>MPileUp</strong> program from the toolbox and select the following settings:  
* *Choose the source for the reference genome*
    * *Use a built-in genome* and
* for the *BAM file(s)*
    * Select the output BAM dataset from the previous step (output of the *MarkDuplicates* tool)
* *Using reference genome* with
    * the *Human reference HG19* selected
* Set *Genotype Likelihood Computation* to
    * *Do not perform genotype likelihood computation (output pileup)* selected 
* Set *Output base positions on reads* to
    * *False*
* Set *Output mapping quality* to
    * *False*
* And finally, *Set advanced options* to
    * *Basic*

**Note:** screenshot below shows the wrong input-file selected, read instructions above.
<img src="pics/MPilup.png">

This tool adds two elements to your history, a log-file that we won't use and a text file in the *pileup* format that we will examine. You can click on the 'eye'-icon to view the contents of this file. Most likely, the first few (tens of) thousands of lines only show data as shown above in the example where only one read mapped. This file can be **very** large, both in size (2 - 3 GB) and in number of lines (100.000.000+ lines). 

The line below is an example where multiple reads (30 in total, as shown in column 4 and by the lengths of columns 5 and 6) mapped on a single position. Here on chromosome 1 we see that in column **5** there is something going on. Read 8 contained a **C** whereas the reference (and the other 29 reads) show a **T** at this position, so this is classified as a mismatch which will be the subject of further analysis steps.
```
chr1	16888646	T	30	.......C........,.,.....,,....	WomHHGomooHmHFHHGGFH1GDHHHHGFF
```
Besides mismatches, we can also have *insertions* and *deletions* such as shown in the example below with a `-` sign followed by a number and a number of bases that are missing. Since we will be working with this data for a while, please read and understand the [pileup format description](https://en.wikipedia.org/wiki/Pileup_format).
```
chr1	16888710	a	31	..,.,..,..,,-2ct,,..,.,,.-2CT,,.,,.,.,,	nGFkGUlFF?DFHFGGEnHFFHHGFHG5GGB
```

### 5.2 Programming Assignments; Coverage Overview for Cardiopanel Genes
 
Somewhere in this enourmous pileup file we can see how (well) our cardiopanel-genes are covered by our reads, a very important statistic on the quality of our data. But in a file with that much data we definitely should not do this manually! This section contains a number of small Python assignments that - once completed - can be combined into a single program that will:
* Load the BED file containing the information (names, chromosome and exon coordinates) of all cardiopanel genes
* Load the `pileup` file containing the mapping data
* For each exon found in the BED file:
    * read the start- and end-coordinate
    * find all entries in the pileup file for this chromosome and within these coordinates
    * for each pileup-entry:
        * store the coverage (data from column 4)
* Given the found coverage for each position in all **exons**:
    * Calculate the average coverage per **gene**
    * Count the number of positions with a coverage < 30
* Write a report on all findings (output to Excel-like file)

Note that by *loading the data* we do not just mean *open* like you would with any other program. In our case we mean to *[parse](https://en.wikipedia.org/wiki/Parsing)* or understand the data so that we can actually use it.

This quite extensive list of tasks shown above describes all the functionality that our program will perform, from reading in data to writing out a report with all calculated coverage statistics. If you really want a challenge or you already have programming experience, the list above and the definitions of the file formats could be sufficient to write your own program without further examples and so-called *templates* that are available below. The list above could be viewed as a lab-protocol defining an experiment and all we need to do now is perform the experiment (in our case; write a program)!

Instead of instructing how to create a new Python program and ask to start programming a solution, we will approach our goal by creating a number of small *units* or in programming terms called *functions* that we will later combine into a functional program, this way, you can focus on a single task. For each of these functions we created a simple *template* Python program that you can download and change given the instructions.

### Deliverable Instructions

Below there are **4** deliverables, for each you are required to carefully read the instructions and - optionally - use a template Python file, and program the missing functionality. Each template consists of two parts:
* One or more functions that you need to edit to add the functionality and
* a `main` function that runs (*calls*) your function with test data and checks the output (a *unit test*)
All of the templates can be executed directly after downloading and they will print some information (that the output is incorrect, obviously). The deliverable is completed once the program prints that the output of your edited function is correct. The fourth deliverable asks to copy the functions from deliverables 1 through 3 into the 4th template to form the complete program.

As a general advice, please evaluate the quality of your code using a *linter* called `pylint`. From the terminal executing `pylint3 deliverable1.py` will scan and *grade* your code. Try to understand what the messages mean and aim for a high grade (a script with a *negative* pylint grade (just imagine) will not be assessed!). 

**Note**: you are *required* to hand in the assignments through a repository **fork**, please read the [instructions](http://nbviewer.jupyter.org/urls/bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/chapters/a2_repository.ipynb) carefully! Do not copy/paste your code into your OneNote journal.

#### Deliverable #1; BED-file parsing

The first step of our program is to read (*parse*) the BED file to get the exon locations for our cardiopanel genes. We already viewed the contents of the BED file in Galaxy, where we saw that this file contains 4 columns:
* The chromosome number, 
* exon start location, 
* exon end location and the
* gene name
```
10	75399683	75399795	MYOZ1  
10	75757946	75758153	VCL  
10	75802821	75802931	VCL  
```

The goal of this program is to parse and store these columns in a *data-structure* so that we can use it once we have the pileup data. The output format, or desired data-structure, is a Python [`dictionary`](https://www.tutorialspoint.com/python/python_dictionary.htm) containing Python [`tuples`](https://www.tutorialspoint.com/python/python_dictionary.htm) with the actual data. For instance, the example with three exons above can be stored as follows:

In [1]:
# Create an empty dictionary that will hold our data
bed_dict = {}
# Add an empty list for the key 10 (chromosome in our case)
bed_dict[10] = []
# Add the tuples with the exon data
bed_dict[10].append((75399683, 75399795, 'MYOZ1'))
bed_dict[10].append((75757946, 75758153, 'VCL'))
bed_dict[10].append((75802821, 75802931, 'VCL'))

# Print the contents
bed_dict

{10: [(75399683, 75399795, 'MYOZ1'),
  (75757946, 75758153, 'VCL'),
  (75802821, 75802931, 'VCL')]}

In this deliverable the BED data needs to be parsed line by line and store data for each exon in a `tuple` object. This `tuple` object can then be `append`-ed to a `list` that is finally stored in a `dictionary` that looks like the one shown in the previous code example.

<!-- MINOR EDIT
The techniques that you will need:
* `for`-loop (iteration)
* `split` (String operations)
* `[0], [1], ...` (list indexing)
* `print`
-->

The template program contains a function called `parse_bed_data(bed_data)` that has one *parameter* (`bed_data`); the contents of the BED file. Download this template and carefully read its instructions: 
[Deliverable 1 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable1.py)

<!-- MINOR EDIT
The techniques that you will need:
* `tuple` (data-structure)
* `if` ([control-flow](https://en.wikipedia.org/wiki/Control_flow))
* `dictionary` (data-structure)
* `append` (list extension)
* `return` (control-flow)
-->

**Challenge**: There is an alternative [Deliverable 1 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable1_OO.py) template script that uses Object Oriented programming that you can try to do (bonus points to be had!). OO-programming will return in a later assignment and is a requirement in later courses too. This challenge does not ask to design the objects yourself but acts as an optional refresher assignment.

#### Deliverable #2; pileup-file parsing and filtering

As with the BED-file parsing, we need to be able to get the relevant column(s) from our file. If we have this data we can use the results from the previous deliverable (the `bed_dict` object with all the exon information) to *filter* out relevant pileup data. For this we are going to *store* the coverage data in a `dictionary`. The template file contains both the input (BED- and pileup-data) and the output data (see the `main` function) that you are working with and is structured as follows:
```
coverage_dict = {
        "RYR2" : [24, 24, 26, 27],
        "DSC2" : [23, 23, 23, 19, 19]
    }
```
where the keys are the gene names and the values a `list` containing a coverage-value for each position.

Template: [Deliverable 2 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable2.py)


#### Deliverable #3; coverage processing

This is the final 'functional' part of our program that we will write which takes the `coverage_dict` from deliverable 2 and calculates a number of values that we are interested in. Given the example data for the `coverage_dict` shown above, we would like to see the following printed when executing the code for this assignment:
```
Gene: RYR2, average coverage: 25.25, low coverage: 4
Gene: DSC2, average coverage: 21.4, low coverage: 5
Total bases: 9, with low coverage:  9 (100%)
```
**Note:** in this case *all* positions have a coverage < 30 (which is what we use as a cutoff value), therefore this deliverable uses a different `coverage_dict` as input, see the template: [Deliverable 3 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable3.py)

There are multiple functions in this deliverable starting with a function that should read a file given its `filename` as input and returns a list with each line as an element (stripped of the newline character). The next function (`save_coverage_statistics`) receives the `coverage_dict` as shown earlier and a `filename` to save the statistics to. Follow the link given in the function so that you are able to use the `csv`-library. The `calculate_mapping_coverage` function eventually builds a list of tuples containing interesting information about each exon. The `main` function is already done and responsible for calling the functions in the correct order (read BED data, read Pileup data, calculate coverage & save to tabular file). 

The product of this deliverable should be a written file that looks **exactly** like this: [Output Data](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/data/d3_example_output.csv). 

**Note:** use the following test-files as input: [BED-data](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/data/example.bed) and [PILEUP-data](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/data/example.pileup). Save these files in the `./data/` folder (the template expects these files to be present and gives an error message if they're missing).

Compare your output file with the example output file and continue if they are the same.

#### Deliverable #4

The complete program! Once the first three deliverables are fully functional, we can now combine all the hard work into a single program that - if we execute it on the BED file and your own pileup file - does all the things we've discussed above. This template contains all the *placeholder* functions that you should replace with the relevant deliverable code, read the instructions carefuly!

Again, only use the example data as downloaded for deliverable 3 and do not yet use your complete pileup-file before you get the OK-signal from the template: [Deliverable 4 Template Python Script](https://bitbucket.org/mkempenaar/diagnosticgenomeanalysis/raw/master/templates/deliverable4.py). Once all code is placed correctly and you are able to run the file you will see something similar to the following output (note; this is run on the complete (100.000.000+ lines) pileup file for patient 2):
```
Reading BED data from Galaxy24-[CAR_0394321__en-20_target_v2.BED].bed
        > A total of 1204 lines have been read.

Reading pileup data from Galaxy27-[MPileup_on_data_26].pileup
        > A total of 106295787 lines have been read.

Parsing BED data...
        > A total of 18 chromosomes have been stored.

Parsing and filtering pileup-data...
        > Coverage of 55 genes have been stored.

Calculating coverage statistics...
        > Statistics for 55 genes have been calculated.

Writing the coverage statistics to d4_output.csv
        > All done!
```


**Download Pileup Data**

Now that you have a working program you should download your own complete pileup file from Galaxy. Right-click the Download button on your pileup-file and use the 'save-as' function to save the file in `/commons/student/2019-2020/`. The template for deliverable 4 allows to provide your own input files on the commandline. This means that you need to:
* go to the folder containing your deliverable 4 program and
* execute your program providing **both** the complete BED- and pileup-files on the commandline (the BED-file is already downloaded for the IGV-assignment):
    * `python3 deliverable4.py ~/folder/with/data.bed /commons/student/2019-2020/Thema05/data.pileup`
    * **Note:** replace the filenames and locations with *your own*!



**Assignment**

This will take a while to process, up to 60 minutes depending on the number of bases sequenced. Upload and view the file in Galaxy and report on what you see:
* Are there genes which might cause concern because of a lot of low-coverage positions?
* Which gene has the most bases sequenced?
* Which gene has the highest percentage of low-coverage bases?
    * Open the IGV-browser again and go to this gene (enter its name in the search field) and visually inspect to determine if the mapping of this gene might give problems. Take a few screenshots (there is a screenshot utility available in the system menu) and - together with these answers - place them in your report.
    